## Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.signal import welch, hilbert
from scipy.stats import skew, kurtosis
import natsort
import networkx as nx

## Functions

In [2]:
def extract_temporal_features(signal):
    """Extract temporal features for a single EEG channel."""
    features = {
        "mean": np.mean(signal),
        "std": np.std(signal),
        "skewness": skew(signal),
        "kurtosis": kurtosis(signal),
        "peak_to_peak": np.ptp(signal),
    }
    return features

def extract_frequency_features(signal, sf=128):
    """Extract frequency features using Welch's method."""
    f, psd = welch(signal, fs=sf, nperseg=sf*2)
    total_power = np.sum(psd)
    features = {
        "abs_delta_power": np.sum(psd[(f >= 0.5) & (f < 4)]),  #  Absolute power in delta band
        "abs_theta_power": np.sum(psd[(f >= 4) & (f < 8)]),  # Absolute power in theta band
        "abs_alpha_power": np.sum(psd[(f >= 8) & (f < 12)]),  # Absolute power in alpha band
        "abs_beta_power": np.sum(psd[(f >= 12) & (f < 30)]),  # Absolute power in beta band
        "rel_gamma_power": np.sum(psd[(f >= 30)]) / total_power,  # Relative power in gamma band
    }
    return features

def compute_visibility_graph(signal, chunk_size=64):  # 64 samples = 0.5 seconds
    """Convert signal to visibility graph and extract graph features."""
    def visibility_graph_chunk(chunk):
        G = nx.Graph()
        n = len(chunk)
        for i in range(n):
            for j in range(i + 1, n):
                k_range = np.arange(i + 1, j)
                if np.all(chunk[k_range] < chunk[i] + (chunk[j] - chunk[i]) * (k_range - i) / (j - i)):
                    G.add_edge(i, j)
        return G

    num_chunks = len(signal) // chunk_size + (1 if len(signal) % chunk_size != 0 else 0)
    all_degrees = []
    all_clustering_coeffs = []
    all_path_lengths = []

    for i in range(num_chunks):
        chunk = signal[i * chunk_size:(i + 1) * chunk_size]
        G = visibility_graph_chunk(chunk)
        degrees = [degree for node, degree in G.degree()]
        all_degrees.extend(degrees)
        all_clustering_coeffs.append(nx.average_clustering(G))
        if nx.is_connected(G):
            all_path_lengths.append(nx.average_shortest_path_length(G))

    avg_degree = np.mean(all_degrees)
    degree_entropy = -np.sum([p * np.log2(p) for p in np.bincount(all_degrees) / len(all_degrees) if p > 0])
    avg_clustering_coeff = np.mean(all_clustering_coeffs)
    avg_path_length = np.mean(all_path_lengths) if all_path_lengths else float('inf')

    features = {
        "avg_degree": avg_degree,
        "degree_entropy": degree_entropy,
        "avg_clustering_coeff": avg_clustering_coeff,
        "avg_path_length": avg_path_length,
    }

    return features

def extract_amplitude_modulation(signal):
    """Extract amplitude modulation features."""
    analytic_signal = hilbert(signal)
    amplitude_envelope = np.abs(analytic_signal)
    features = {
        "am_mean": np.mean(amplitude_envelope),
        "am_std": np.std(amplitude_envelope),
    }
    return features




## Features extraction

In [3]:
# Define directories
base_dir = "Data/Preprocessed"
activities = ["Arithmetic", "Mirror_image", "Relax", "Stroop"]

# Main processing loop
results = []

for activity in activities:
    print(f"Processing activity: {activity}")
    activity_dir = os.path.join(base_dir, activity)
    for i, file in enumerate(natsort.natsorted(os.listdir(activity_dir))):
        if file.endswith(".csv"):
            if i % 3 == 0:
                print(".", end="", flush=True)
            file_path = os.path.join(activity_dir, file)
            data = pd.read_csv(file_path)

            # Extract time points and EEG signals
            time = data.iloc[:, 0]
            signals = data.iloc[:, 1:].values  # All EEG channels
            
            for channel_idx, signal in enumerate(signals.T):
                # Combine all feature extraction
                temporal_features = extract_temporal_features(signal)
                frequency_features = extract_frequency_features(signal)
                vg_features = compute_visibility_graph(signal)
                am_features = extract_amplitude_modulation(signal)
                
                # Merge features
                features = {
                    "activity": activity,
                    "file": file,
                    "channel": f"channel_{channel_idx + 1}",
                    **temporal_features,
                    **frequency_features,
                    **vg_features,
                    **am_features,
                }
                results.append(features)
    print()  # New line after each activity


Processing activity: Arithmetic
........................................
Processing activity: Mirror_image
........................................
Processing activity: Relax
........................................
Processing activity: Stroop
........................................


## Save features

In [4]:
# Save results to CSV
features_df = pd.DataFrame(results)
features_df.to_csv("eeg_features.csv", index=False)
print("Feature extraction complete. Results saved to eeg_features.csv.")

Feature extraction complete. Results saved to eeg_features.csv.
